In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/9월/나무문화_9월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

389


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_9월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,0,"['50', 'm', 'NAVER', 'Corp', '송천', '도서관', '전라북...","[('50', 'Number'), ('m', 'Alpha'), ('NAVER', '..."
1,강사는 불편한편의점 의 김호연 작가이다 불편한 편의점 2 저자 김호연 출판 나무옆의...,0,"['강사', '는', '불편한', '편의점', '의', '김호연', '작가', '이...","[('강사', 'Noun'), ('는', 'Josa'), ('불편하다', 'Adje..."
2,주제 소설가의 삶과 문화콘텐츠 스토리텔링 강의 시작전 22년간 영화 만화 출판 관련...,0,"['주제', '소설가', '의', '삶', '과', '문화', '콘텐츠', '스토리...","[('주제', 'Noun'), ('소설가', 'Noun'), ('의', 'Josa'..."
3,문화콘텐츠 문화콘텐츠란 인간의 감성 창의성 상상력을 원천으로 한 문화상품 문화콘텐츠...,0,"['문화', '콘텐츠', '문화', '콘텐츠', '란', '인간', '의', '감성...","[('문화', 'Noun'), ('콘텐츠', 'Noun'), ('문화', 'Noun..."
4,영화 드라마 만화 대중음악 뮤지컬 모바일게임 인터넷 모바일 테마파크 전시 축제 공간...,0,"['영화', '드라마', '만화', '대중음악', '뮤지컬', '모바일', '게임'...","[('영화', 'Noun'), ('드라마', 'Noun'), ('만화', 'Noun..."
...,...,...,...,...
1021052,라벨에 BIO라는 단어가 지워져 있습니다,20813,"['라벨', '에', 'BIO', '라는', '단어', '가', '지워져', '있습...","[('라벨', 'Noun'), ('에', 'Josa'), ('BIO', 'Alpha..."
1021053,원재료의 일부로 들어간 유기농 설탕과 시나몬이 유럽에서는 유기농 인증서가 있음에도 ...,20813,"['원', '재료', '의', '일부', '로', '들어간', '유기농', '설탕'...","[('원', 'Modifier'), ('재료', 'Noun'), ('의', 'Jos..."
1021054,아버지를 아버지라 부르지 못하는 홍길동처럼 유기농을 유기농이라 부르지 못하는 홍길동...,20813,"['아버지', '를', '아버지', '라', '부르지', '못', '하는', '홍길...","[('아버지', 'Noun'), ('를', 'Josa'), ('아버지', 'Noun..."
1021055,무더운 여름 건강 조심하십시요,20813,"['무더운', '여름', '건강', '조심하십시요']","[('무덥다', 'Adjective'), ('여름', 'Noun'), ('건강', ..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,0,"['50', 'm', 'NAVER', 'Corp', '송천', '도서관', '전라북...","[('50', 'Number'), ('m', 'Alpha'), ('NAVER', '...",50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
1,강사는 불편한편의점 의 김호연 작가이다 불편한 편의점 2 저자 김호연 출판 나무옆의...,0,"['강사', '는', '불편한', '편의점', '의', '김호연', '작가', '이...","[('강사', 'Noun'), ('는', 'Josa'), ('불편하다', 'Adje...",강사 는 불편한 편의점 의 김호연 작가 이다 불편한 편의점 2 저자 김호연 출판 나...
2,주제 소설가의 삶과 문화콘텐츠 스토리텔링 강의 시작전 22년간 영화 만화 출판 관련...,0,"['주제', '소설가', '의', '삶', '과', '문화', '콘텐츠', '스토리...","[('주제', 'Noun'), ('소설가', 'Noun'), ('의', 'Josa'...",주제 소설가 의 삶 과 문화 콘텐츠 스토리텔링 강의 시 작전 22년 간 영화 만화 ...
3,문화콘텐츠 문화콘텐츠란 인간의 감성 창의성 상상력을 원천으로 한 문화상품 문화콘텐츠...,0,"['문화', '콘텐츠', '문화', '콘텐츠', '란', '인간', '의', '감성...","[('문화', 'Noun'), ('콘텐츠', 'Noun'), ('문화', 'Noun...",문화 콘텐츠 문화 콘텐츠 란 인간 의 감성 창의성 상상력 을 원천 으로 한 문화 상...
4,영화 드라마 만화 대중음악 뮤지컬 모바일게임 인터넷 모바일 테마파크 전시 축제 공간...,0,"['영화', '드라마', '만화', '대중음악', '뮤지컬', '모바일', '게임'...","[('영화', 'Noun'), ('드라마', 'Noun'), ('만화', 'Noun...",영화 드라마 만화 대중음악 뮤지컬 모바일 게임 인터넷 모바일 테마 파크 전시 축제 ...
...,...,...,...,...,...
1021052,라벨에 BIO라는 단어가 지워져 있습니다,20813,"['라벨', '에', 'BIO', '라는', '단어', '가', '지워져', '있습...","[('라벨', 'Noun'), ('에', 'Josa'), ('BIO', 'Alpha...",라벨 에 BIO 라는 단어 가 지워져 있습니다
1021053,원재료의 일부로 들어간 유기농 설탕과 시나몬이 유럽에서는 유기농 인증서가 있음에도 ...,20813,"['원', '재료', '의', '일부', '로', '들어간', '유기농', '설탕'...","[('원', 'Modifier'), ('재료', 'Noun'), ('의', 'Jos...",원 재료 의 일부 로 들어간 유기농 설탕 과 시나몬 이 유럽 에서는 유기농 인증서 ...
1021054,아버지를 아버지라 부르지 못하는 홍길동처럼 유기농을 유기농이라 부르지 못하는 홍길동...,20813,"['아버지', '를', '아버지', '라', '부르지', '못', '하는', '홍길...","[('아버지', 'Noun'), ('를', 'Josa'), ('아버지', 'Noun...",아버지 를 아버지 라 부르지 못 하는 홍길동 처럼 유기농 을 유기농 이라 부르지 못...
1021055,무더운 여름 건강 조심하십시요,20813,"['무더운', '여름', '건강', '조심하십시요']","[('무덥다', 'Adjective'), ('여름', 'Noun'), ('건강', ...",무더운 여름 건강 조심하십시요


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,50m NAVER Corp 송천도서관 전라북도 전주시 덕진구 솔내로 212 송천도서...,0,"['50', 'm', 'NAVER', 'Corp', '송천', '도서관', '전라북...","[('50', 'Number'), ('m', 'Alpha'), ('NAVER', '...",50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
1,강사는 불편한편의점 의 김호연 작가이다 불편한 편의점 2 저자 김호연 출판 나무옆의...,0,"['강사', '는', '불편한', '편의점', '의', '김호연', '작가', '이...","[('강사', 'Noun'), ('는', 'Josa'), ('불편하다', 'Adje...",강사 는 불편한 편의점 의 김호연 작가 이다 불편한 편의점 2 저자 김호연 출판 나...,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
2,주제 소설가의 삶과 문화콘텐츠 스토리텔링 강의 시작전 22년간 영화 만화 출판 관련...,0,"['주제', '소설가', '의', '삶', '과', '문화', '콘텐츠', '스토리...","[('주제', 'Noun'), ('소설가', 'Noun'), ('의', 'Josa'...",주제 소설가 의 삶 과 문화 콘텐츠 스토리텔링 강의 시 작전 22년 간 영화 만화 ...,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
3,문화콘텐츠 문화콘텐츠란 인간의 감성 창의성 상상력을 원천으로 한 문화상품 문화콘텐츠...,0,"['문화', '콘텐츠', '문화', '콘텐츠', '란', '인간', '의', '감성...","[('문화', 'Noun'), ('콘텐츠', 'Noun'), ('문화', 'Noun...",문화 콘텐츠 문화 콘텐츠 란 인간 의 감성 창의성 상상력 을 원천 으로 한 문화 상...,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
4,영화 드라마 만화 대중음악 뮤지컬 모바일게임 인터넷 모바일 테마파크 전시 축제 공간...,0,"['영화', '드라마', '만화', '대중음악', '뮤지컬', '모바일', '게임'...","[('영화', 'Noun'), ('드라마', 'Noun'), ('만화', 'Noun...",영화 드라마 만화 대중음악 뮤지컬 모바일 게임 인터넷 모바일 테마 파크 전시 축제 ...,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
...,...,...,...,...,...,...
1021052,라벨에 BIO라는 단어가 지워져 있습니다,20813,"['라벨', '에', 'BIO', '라는', '단어', '가', '지워져', '있습...","[('라벨', 'Noun'), ('에', 'Josa'), ('BIO', 'Alpha...",라벨 에 BIO 라는 단어 가 지워져 있습니다,2022 바캉스 와 한 여름 에 어울리는 낭만 와인 들 출처 http www fre...
1021053,원재료의 일부로 들어간 유기농 설탕과 시나몬이 유럽에서는 유기농 인증서가 있음에도 ...,20813,"['원', '재료', '의', '일부', '로', '들어간', '유기농', '설탕'...","[('원', 'Modifier'), ('재료', 'Noun'), ('의', 'Jos...",원 재료 의 일부 로 들어간 유기농 설탕 과 시나몬 이 유럽 에서는 유기농 인증서 ...,2022 바캉스 와 한 여름 에 어울리는 낭만 와인 들 출처 http www fre...
1021054,아버지를 아버지라 부르지 못하는 홍길동처럼 유기농을 유기농이라 부르지 못하는 홍길동...,20813,"['아버지', '를', '아버지', '라', '부르지', '못', '하는', '홍길...","[('아버지', 'Noun'), ('를', 'Josa'), ('아버지', 'Noun...",아버지 를 아버지 라 부르지 못 하는 홍길동 처럼 유기농 을 유기농 이라 부르지 못...,2022 바캉스 와 한 여름 에 어울리는 낭만 와인 들 출처 http www fre...
1021055,무더운 여름 건강 조심하십시요,20813,"['무더운', '여름', '건강', '조심하십시요']","[('무덥다', 'Adjective'), ('여름', 'Noun'), ('건강', ...",무더운 여름 건강 조심하십시요,2022 바캉스 와 한 여름 에 어울리는 낭만 와인 들 출처 http www fre...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,50 m NAVER Corp 송천 도서관 전라북도 전주시 덕진구 솔 내 로 212 ...
1,1,즐거운 책 놀이터 독서 교육 강사 이숙영 입니다 어느 새 무더운 여름 이 지나고 9...
2,2,다시 짧은 시간 이 지났고 바람 이 불었고 그렇게 앉아있는 동안 나 는 내 가 이 ...
3,3,멜크 수도원 멜크 수도원 2014년 4월 22일 2014년 5월 2일 베네룩스 3 ...
4,4,추석 연휴 놀러 갈만 한 곳 강원도 여행 평창 청옥산 육 백 마지기 글 사진 초록 ...
...,...,...
20808,20809,안녕하세요 쌍대 킴 입니다 오늘 은 여름 휴가 를 맞이 하여 찐 맛집 블로거 모드 ...
20809,20810,강릉 여행 오죽헌 feat 시립 박물관 강릉 에 왔다면 꼭 들려야 한다는 오죽헌 강...
20810,20811,주소 대전 유성구 문화 원로 41 영업 시간 매주 월요일 휴무 비 가 많이 내린 일...
20811,20812,자세한 줄거리 결말 포함 스포 주의 본 내용 은 감독판 을 기반 하여 작성 되었음 ...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(20813, 472388)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.02604313, 0.07196596, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,472378,472379,472380,472381,472382,472383,472384,472385,472386,472387
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20808,0.014635,0.047181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20809,0.000000,0.049994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20810,0.026043,0.071966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20811,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,00010,0001047477,0001152252,0001154435,0001157693,0001169504,...,힝,힝구,힝굴,힝규,힝기스,힝깅,힝행훙,힝힝,힡,힣
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20808,0.014635,0.047181,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20809,0.000000,0.049994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20810,0.026043,0.071966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20811,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/9월/나무문화_9월_TF-IDF행렬(원본).csv", index=False, encoding='UTF-8')

In [15]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

         ㄱ   ㄱㄱ  ㄱㄱㄱ  ㄱㄱㅅ  ㄱㄱㅆ  ㄱㄱㅇ  ㄱㄱㅊ  ㄱㄱㅑ   ㄱㄴ  ㄱㄴㄷ  ...    힝   힝구   힝굴  \
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
20808  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
20809  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
20810  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
20811  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
20812  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

        힝규  힝기스   힝깅  힝행훙   힝힝    힡    힣  
0      0

In [16]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,108.626675
1,00,291.970607
2,000,39.639210
3,0001,0.579976
4,00010,0.020988
...,...,...
472383,힝깅,0.039948
472384,힝행훙,0.032518
472385,힝힝,0.177545
472386,힡,0.074727


In [17]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/9월/나무문화_9월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')